In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Reading data

In [2]:
data_file_path = '../data/MELBOURNE_HOUSE_PRICES_LESS.csv'

mhp_data = pd.read_csv(data_file_path)
# keep only "Property Sold" and "Property Sold Prior"
mhp_data = mhp_data[mhp_data.Method.isin(['S', 'SP'])]

mhp_data = mhp_data[mhp_data.Price.notnull()]
mhp_data.describe()

,Rooms,Price,Postcode,Propertycount,Distance
count,37104.000000,3.710400e+04,37104.000000,37104.000000,37104.000000
mean,3.046572,9.626983e+05,3124.967227,7590.386778,13.168995
std,0.920345,5.511464e+05,132.916619,4499.211539,7.671679
min,1.000000,1.120000e+05,3000.000000,83.000000,0.000000
25%,2.000000,6.105000e+05,3048.000000,4242.000000,7.200000
50%,3.000000,8.100000e+05,3095.000000,6567.000000,12.100000
75%,4.000000,1.180000e+06,3163.000000,10412.000000,17.300000
max,31.000000,7.650000e+06,3980.000000,21650.000000,55.800000


In [3]:
mhp_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council


In [4]:
from sklearn.model_selection import train_test_split

mhp_train_set, mhp_val_set = train_test_split(mhp_data, test_size = 0.2, random_state = 111)

## Tranforming Data

In [5]:
mph_num = ['Rooms', 'Distance', 'Propertycount']
mph_cat = ['Type', 'CouncilArea']

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

class ToFloatConverter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.astype(float)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_pipeline = Pipeline([
    ('to_float', ToFloatConverter()),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, mph_num),
    ('cat', cat_pipeline, mph_cat),
])

train_X = full_pipeline.fit_transform(mhp_train_set)
train_y = mhp_train_set.Price
train_X.shape

(29683, 39)

In [8]:
val_X = full_pipeline.transform(mhp_val_set)
val_y = mhp_val_set.Price
val_X.shape

(7421, 39)

## Learning

In [9]:
from sklearn.metrics import mean_squared_error

def get_rmse(model, X, y):
    predictions = model.predict(X)
    mse = mean_squared_error(y, predictions)
    return np.sqrt(mse)

In [10]:
def print_rmse(name, model, train_X, train_y, val_X, val_y):
    print(name, 'train RMSE', get_rmse(model, train_X, train_y))
    print(name, 'validation RMSE', get_rmse(model, val_X, val_y))

In [12]:
def print_rmse_grid(grid):
    for mean_score, params in zip(grid.cv_results_["mean_test_score"], grid.cv_results_["params"]):
        print(np.sqrt(-mean_score), params)

### Random Forest

In [13]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state = 111, n_estimators = 10)
forest_reg.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=111, verbose=0, warm_start=False)

In [14]:
print_rmse('Random Forest', forest_reg, train_X, train_y, val_X, val_y)

Random Forest train RMSE 258141.35677966513
Random Forest validation RMSE 271380.96816115326


### Decision Tree

In [15]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state = 111)
tree_reg.fit(train_X, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=111, splitter='best')

In [16]:
print_rmse('Decision Tree', tree_reg, train_X, train_y, val_X, val_y)

Decision Tree train RMSE 255874.8995604695
Decision Tree validation RMSE 277695.4962710435


#### GridSearch with DecisionTree

In [17]:
from sklearn.model_selection import GridSearchCV

tree_params_grid = {'max_leaf_nodes': [None, 128, 256, 512, 1024, 2048]}

tree_reg_grid = DecisionTreeRegressor(random_state = 111)

tree_grid_search = GridSearchCV(tree_reg_grid, tree_params_grid, cv=5, 
                                scoring='neg_mean_squared_error', return_train_score=True,
                                verbose=2, n_jobs=-1)
tree_grid_search.fit(train_X, train_y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    3.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=111, splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_leaf_nodes': [None, 128, 256, 512, 1024, 2048]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=2)

In [18]:
print_rmse('Grid Search with Decision Tree', tree_grid_search.best_estimator_, train_X, train_y, val_X, val_y)

Grid Search with Decision Tree train RMSE 257502.7158352101
Grid Search with Decision Tree validation RMSE 278058.3745598735


In [19]:
tree_grid_search.best_params_

{'max_leaf_nodes': 1024}

In [20]:
print_rmse_grid(tree_grid_search)

291674.78461384255 {'max_leaf_nodes': None}
311764.3010200495 {'max_leaf_nodes': 128}
299246.2260283386 {'max_leaf_nodes': 256}
293040.509493982 {'max_leaf_nodes': 512}
290966.0189841305 {'max_leaf_nodes': 1024}
291046.36174789193 {'max_leaf_nodes': 2048}


#### RandomizedSearch with Decision Tree

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

tree_params_rnd = {'max_leaf_nodes': sp_randint(100, 5000)}

tree_reg_rnd = DecisionTreeRegressor(random_state = 111)

tree_rnd_search = RandomizedSearchCV(tree_reg_rnd, tree_params_rnd, cv=5, n_iter=20,n_jobs=-1)
tree_rnd_search.fit(train_X, train_y)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=111, splitter='best'),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'max_leaf_nodes': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00FB4810>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [24]:
print_rmse('Randomized Search with Decision Tree', tree_rnd_search.best_estimator_, train_X, train_y, val_X, val_y)

Randomized Search with Decision Tree train RMSE 256892.23450729693
Randomized Search with Decision Tree validation RMSE 277799.10570831684


In [25]:
tree_rnd_search.best_params_

{'max_leaf_nodes': 1176}

### SVM

In [27]:
from sklearn.svm import SVR

svm_reg = SVR(gamma='auto')
svm_reg.fit(train_X, train_y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [28]:
print_rmse('SVM', svm_reg, train_X, train_y, val_X, val_y)

SVM train RMSE 575608.9651023118
SVM validation RMSE 555824.0090454625


#### GridSearch with SVM

In [29]:
from sklearn.model_selection import GridSearchCV

svm_params_grid = [{'kernel': ['rbf', 'sigmoid'], 'gamma': ['auto', 'scale'], 'C': [0.1, 1.0, 10.0]},
                   {'kernel': ['linear'], 'C': [0.1, 1.0, 10.0]}]

svm_reg_grid = SVR(gamma='auto')
svm_grid_search = GridSearchCV(svm_reg_grid, svm_params_grid, cv=5, 
                               scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)
svm_grid_search.fit(train_X, train_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'kernel': ['rbf', 'sigmoid'], 'gamma': ['auto', 'scale'], 'C': [0.1, 1.0, 10.0]}, {'kernel': ['linear'], 'C': [0.1, 1.0, 10.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [30]:
print_rmse('Grid Search with SVM', svm_grid_search.best_estimator_, train_X, train_y, val_X, val_y)

Grid Search with SVM train RMSE 529071.8841789505
Grid Search with SVM validation RMSE 509102.49260402715


In [31]:
svm_grid_search.best_params_

{'C': 10.0, 'kernel': 'linear'}

In [32]:
print_rmse_grid(svm_grid_search)

576061.8430194232 {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
576069.8621208319 {'C': 0.1, 'gamma': 'auto', 'kernel': 'sigmoid'}
576041.8666409021 {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
576044.7442643257 {'C': 0.1, 'gamma': 'scale', 'kernel': 'sigmoid'}
575860.2211544343 {'C': 1.0, 'gamma': 'auto', 'kernel': 'rbf'}
575940.3070444165 {'C': 1.0, 'gamma': 'auto', 'kernel': 'sigmoid'}
575672.4471732406 {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'}
575696.9926690961 {'C': 1.0, 'gamma': 'scale', 'kernel': 'sigmoid'}
574007.6504314088 {'C': 10.0, 'gamma': 'auto', 'kernel': 'rbf'}
574752.6948986297 {'C': 10.0, 'gamma': 'auto', 'kernel': 'sigmoid'}
572225.2864746623 {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}
572431.7930309661 {'C': 10.0, 'gamma': 'scale', 'kernel': 'sigmoid'}
575544.52649705 {'C': 0.1, 'kernel': 'linear'}
570964.7995237166 {'C': 1.0, 'kernel': 'linear'}
535693.3682345026 {'C': 10.0, 'kernel': 'linear'}


#### RandomizedSearch with SVM

In [33]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

svm_params_rnd = {'kernel': ['linear'], 'C': sp_randint(1.0, 1000.0)}

svm_reg_rnd = SVR(gamma='auto')
svm_reg_rnd = RandomizedSearchCV(svm_reg_rnd, svm_params_rnd, cv=5, n_iter=20, n_jobs=-1)
svm_reg_rnd.fit(train_X, train_y)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'kernel': ['linear'], 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x04DD5270>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [34]:
print_rmse('Randomized Search with SVM', svm_reg_rnd.best_estimator_, train_X, train_y, val_X, val_y)

Randomized Search with SVM train RMSE 398565.8776385977
Randomized Search with SVM validation RMSE 379606.16862571385


In [35]:
svm_reg_rnd.best_params_

{'C': 836, 'kernel': 'linear'}